#Select VASCA sources based on chosen criteria

In [1]:
region_name = "ALL_10-800" #"CAINGSGII_10-800" #"TDS" #"CAINGSGII_10-800" #"WD" #"MDIS_10-800" #"TDS" #  _ELAISN1
region_cat_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+"_cat.fits"

### Prepare tt_sources with variables for selection

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table
from vasca.region import Region
import vasca.visualization as vvis
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, get_col_cycler,add_ogrp, color_palette, nb_fig
from astropy import units as uu
from astropy.table import Column

# Get region
rc = Region()
rc.load_from_fits(region_cat_fname)
rc.tt_sources.sort(["dec"])

#Add Object group to source table
rc.copy_table_columns("tt_sources","tt_simbad",["otype","ogrp","match_distance","sp_type","main_id"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_simbad")
rc.tt_sources.rename_column("ogrp","ogrp_simbad")
sel_simbad = rc.tt_sources["sel"].data
print(f"Number of SIMBAD matches is: {sel_simbad.sum()}  ({100* sel_simbad.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_gaiadr3",["PQSO", "PGal", "PSS","match_distance","Gmag_abs","BP-RP", "ogrp", "Plx","e_Plx","Plx_dist","VarFlag","RPlx"],"rg_src_id")
rc.tt_sources.rename_column("match_distance","match_distance_gaiadr3")
rc.tt_sources.rename_column("ogrp","ogrp_gaiadr3")
sel_gaiadr3 = rc.tt_sources["sel"].data
print(f"Number of GAIA matches is: {sel_gaiadr3.sum()}  ({100*sel_gaiadr3.sum() / len(rc.tt_sources)} )%")

rc.copy_table_columns("tt_sources","tt_maincat",["Pwd"],"rg_src_id")
sel_gaia2wd = rc.tt_sources["sel"].data
print(f"Number of GAIA WD matches is: {sel_gaia2wd.sum()}  ({100*sel_gaia2wd.sum() / len(rc.tt_sources)} )%")

sel_match = np.array(sel_simbad+sel_gaiadr3, dtype=bool)
print(f"Total number of matches is: {sel_match.sum()}  ({100*sel_match.sum() / len(rc.tt_sources)} )%")

#Copy lomb scargle result into tt_sources
rc.copy_table_columns("tt_sources","tt_lombscargle",["ls_peak_pval", "ls_model_rchiq"],"rg_src_id")

display(rc.tt_simbad[rc.tt_simbad["ogrp"]=="WD*"])
display(rc.tt_gaiadr3)

Number of SIMBAD matches is: 4761  (56.84097421203438 )%
Number of GAIA matches is: 6216  (74.21203438395415 )%
Number of GAIA WD matches is: 22  (0.2626552053486151 )%
Total number of matches is: 6885  (82.19914040114614 )%


main_id,ra,dec,ra_prec,dec_prec,coo_err_maja,coo_err_mina,coo_err_angle,coo_qual,coo_wavelength,coo_bibcode,otype,otypes,distance_distance,distance_q,distance_unit,distance_merr,distance_perr,distance_method,distance_bibcode,match_distance,rvz_type,rvz_radvel,rvz_error,rvz_qual,rvz_wavelength,rvz_bibcode,z_value,sp_type,sp_qual,sp_bibcode,script_number_id,rg_src_id,simbad_match_id,ogrp
,deg,deg,,,mas,mas,deg,,,,,,,,,,,,,arcsec,,km / s,km / s,,,,,,,,,,,
bytes32,float64,float64,int16,int16,float32,float32,int16,bytes1,bytes1,bytes32,bytes32,bytes32,float64,bytes1,bytes4,float64,float64,bytes8,bytes19,float32,bytes1,float64,float32,bytes1,bytes1,bytes32,float64,bytes32,bytes32,bytes32,int32,int32,int32,bytes8
V* V2276 Sgr,306.59244125,-43.6754825,14,14,0.331,0.248,90,A,O,2020yCat.1350....0G,CV*,CV*|V*|*|WD?,--,--,--,--,--,--,--,0.2714,--,--,--,--,--,--,--,--,--,--,348,21770,267,WD*
SBSS 1612+554,243.31930499999993,55.357211944444444,14,14,0.031,0.028,90,A,O,2020yCat.1350....0G,WD*,WD*|Pu*|MIR|NIR|WD*|*|Opt|WD?|WD,65.9420,--,pc,-0.1309,0.1309,paral,2020yCat.1350....0G,0.1378,--,--,--,--,--,--,--,DA4.2,C,2011ApJ...743..138G,395,25273,288,WD*
GALEX J161003.8+561350,242.51598291666667,56.23088694444445,14,14,0.128,0.121,90,A,O,2020yCat.1350....0G,WD?,*|UV|WD?|WD?,--,--,--,--,--,--,--,0.3921,--,--,--,--,--,--,--,--,--,--,401,26195,295,WD*
V* IY UMa,160.98645291666665,58.12553333333334,14,14,0.036,0.046,90,A,O,2020yCat.1350....0G,CV*,PM*|CV*|CV*|V*|PM*|NIR|*|*|CV?|C,182.0470,--,pc,-2.2337,2.2337,paral,2020yCat.1350....0G,0.3422,z,0.001,0.000,E,--,2009ApJS..182..543A,0.0005300,--,--,--,494,32448,350,WD*
2QZ J100132.7+013057,150.38664125,1.5162241666666667,14,14,0.364,0.397,90,A,O,2020yCat.1350....0G,WD*,WD*|WD*|blu|*|*|G|Opt|UV|WD?,1110.4600,--,pc,-129.4600,129.4600,ST-L,2017MNRAS.469.2102A,0.5794,v,-74.400,19.330,C,O,2017MNRAS.469.2102A,-0.0002480,DA,D,2015MNRAS.446.4078K,541,34658,390,WD*
M31N 2008-05d,11.007374999999998,41.07325,6,6,--,--,0,C,O,2008ATel.1602....1H,No*,No*,--,--,--,--,--,--,--,0.5632,--,--,--,--,--,--,--,--,--,--,858,53048,581,WD*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V* MQ Dra,238.37966583333332,55.2706875,14,14,0.055,0.066,90,A,O,2020yCat.1350....0G,CV*,EB*|V*|CV*|V*|CV*|NIR|*|Opt|X,186.2340,--,pc,-3.2116,3.2116,paral,2018yCat.1345....0G,0.2065,z,-0.000,0.000,E,--,2008ApJS..175..297A,-0.0003500,M5Ve,D,2003ApJ...583..902S,6442,1232731,3758,WD*


_q,ra,dec,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,PQSO,PGal,PSS,RPlx,VarFlag,o_Gmag,rg_src_id,match_distance,gaiadr3_match_id,Gmag_abs,Plx_dist,ogrp
,deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,,,,,,,arcsec,,,pc,
int32,float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,bytes12,bytes15,bytes19,int32,bytes10,bytes16,bytes17,bytes21,float64,float64,float64,float64,float64,float64,bytes13,int16,int32,float64,int32,float32,float64,bytes8
1,150.2,2.1,3836069006998531584,2.2872,3.3137,--,--,--,--,--,--,--,--,95.82888,1.689,20.733625,93.75,16.38,20.408556,130.9,9.985,19.455320,0.953236,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110481501947501974,1237651753997304055,--,--,--,J100046.74+020404.4,--,N6SS000981,--,--,--,150.2,2.1,0.001869,0.000724,0.996765,--,NOT_AVAILABLE,79,108,0.704386779266328,0,-100.0,--,none
2,150.2,2.1,3836070076445471744,3.6095,4.1203,--,--,--,--,--,--,--,--,146.36924,3.992,20.273743,185.5,14.67,19.667881,286.5,18.95,18.604939,1.062943,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,--,1237651753997303903,--,--,--,J100043.14+020637.1,--,N6SS001075,--,--,--,150.2,2.1,0.000287,0.000160,0.998952,--,NOT_AVAILABLE,109,120,0.8139196155304314,1,-100.0,--,none
3,150.1,2.0,3836046123412862208,1.5776,2.0453,--,--,--,--,--,--,--,--,90.12499,2.676,20.800253,37.12,20.52,21.414429,59.56,10.83,20.310591,1.103838,--,--,--,--,0,0,0,0,0,0,0,0,0,0,0,--,--,--,--,--,--,110451501456232172,1237651736280760581,--,--,--,J100034.94+020234.9,--,N6SS004829,--,--,--,150.1,2.0,0.001265,0.000013,0.997987,--,NOT_AVAILABLE,43,154,0.5969659350492861,2,-100.0,--,none
5,150.1,2.1,3836257710681556224,0.5228,0.5439,-0.0737,0.5331,0.702,-0.580,0.607,0.396,0.641,1.065,231.69009,2.456,19.775099,160.4,10.57,19.825287,132.1,14.81,19.445589,0.379698,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,110521501020297033,1237651753997238486,--,--,--,J100024.44+020619.5,--,N6SV003245,--,--,--,150.1,2.1,1.000000,0.000000,0.000000,-0.1383,VARIABLE,182,187,0.8013254227793969,3,-100.0,--,AGN
9,215.1,52.6,1607957522794289024,0.1805,0.2101,-0.4327,0.2279,0.356,0.190,0.223,0.302,0.256,1.036,311.79894,1.36,19.452681,178.4,9.809,19.709816,189.3,7.18,19.055189,0.654627,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171142150651746989,1237661385994862739,--,--,--,J142015.62+523718.5,--,N5MV001218,--,--,--,215.1,52.6,1.000000,0.000000,0.000000,-1.8990,VARIABLE,482,512,0.2679557969426055,4,-100.0,--,AGN
10,215.0,52.7,1607958480571152256,0.8988,0.7715,-1.1739,0.6843,0.532,-0.092,1.013,0.524,0.894,1.079,127.05163,2.726,20.427416,98.51,10.98,20.354832,127.2,7.357,19.486479,0.868353,--,--,--,--,1,0,0,0,0,0,1,0,0,0,0,--,--,--,--,--,--,171212150426430532,1237661385994862858,--,--,--,J142010.22+524029.6,--,N5MV001750,--,--,--,215.0,52.7,0.999765,0.000220,0.000015,-1.7155,VARIABLE,283,524,0.5654019828574242,5,-100.0,--,AGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8370,5.0,17.1,2793606540757272064,0.0454,0.0420,3.7128,0.0613,18.094,7.430,0.057,16.497,0.060,2.945,1924799.21828,341.7,9.976403,1.106e+06,285.9,10.228971,1.189e+06,227.2,9.55

### Print info for selected sources

In [5]:
# Make selection
sel_otype = rc.tt_sources["otype"] ==  "RR*"# "No*"#"HS?" #"BS*" #"PM*" # # "SN*" #"EB*"
sel_otypes =   (rc.tt_sources["otype"] =="WD*")*(rc.tt_sources["sp_type"] !="none") +(rc.tt_sources["Pwd"] >0.9) # (rc.tt_sources["otype"] ==  "WD?")  + # 
sel_flux_nxv = rc.tt_sources["flux_nxv"][:,0]>2
sel_flux_hr = rc.tt_sources["hr"]>2
sel_src_id = rc.tt_sources["rg_src_id"]==15179
sel_period = (rc.tt_sources["ls_peak_pval"]>-0.5) * (rc.tt_sources["ls_peak_pval"]<0.000000573303) * (rc.tt_sources["ls_model_rchiq"]<1.1)# # 3 sigma 0.002699796063, 4 sigma 0.000063342484, 5 sigma 0.000000573303
sel_srcs =  sel_otypes #sel_period #sel_otypes  #sel_src_id  #sel_flux_nxv #sel_otype # sel_flux_nxv_fuv # sel_flux_hr#

#Print oout list of source IDs to be used for further use, e.g. in inspect_sources.ipynb
print("\nsrcs_ids = [", end="")
for src_id in rc.tt_sources[sel_srcs]["rg_src_id"].data:
    print(src_id,",", end="")
print("]\n")
print("Total Nr. of sources:",sel_srcs.sum(),"\n")

#Display match and simbad table for selected sources
cols_write = ["ra","dec","ogrp_gaiadr3","otype","sp_type","main_id","nr_det","flux","flux_cpval","flux_ne","Plx_dist","ls_peak_pval","ls_model_rchiq","VarFlag","Pwd","RPlx","rg_src_id"]
display(rc.tt_sources[sel_srcs][cols_write] )#["sp_type","main_id"]
rc.tt_sources[sel_srcs][cols_write].write("sources.ecsv",overwrite=True,delimiter=',')


srcs_ids = [535864 ,451644 ,541266 ,581995 ,625693 ,187856 ,494782 ,166179 ,172775 ,34658 ,1521738 ,426363 ,151796 ,259271 ,419001 ,25273 ,]

Total Nr. of sources: 16 



ra,dec,ogrp_gaiadr3,otype,sp_type,main_id,nr_det,flux,flux_cpval,flux_ne,Plx_dist,ls_peak_pval,ls_model_rchiq,VarFlag,Pwd,RPlx,rg_src_id
deg,deg,,,,,,1e-06 Jy,,,pc,,,,,,
float64,float64,bytes8,bytes32,bytes32,bytes32,int32[2],float32[2],float32[2],float32[2],float32,float32,float32,bytes32,float32,float32,int32
17.908561463231784,-46.124209856672074,WD*,none,none,none,12 .. 2,210.91878 .. 51.06775,4.861283e-20 .. 0.8805627,0.05030113 .. --,152.56229,-1.0,-1.0,VARIABLE,0.988864,93.8769,535864
53.603003894660404,-35.35703047345705,WD*,WD?,,GALEX J033424.7-352125,24 .. 28,309.9156 .. 470.0984,0.106752574 .. 9.816341e-09,0.011271525 .. 0.025724309,430.47784,0.5929085,0.87381697,NOT_AVAILABLE,0.99926,15.7304,451644
12.321346896575124,-25.432354111852234,WD*,WD*,DA,GALEX J004917.1-252556,11 .. 1,73.65471 .. 41.309494,1.0632669e-08 .. -1.0,0.04259176 .. --,99.60556,-1.0,-1.0,NOT_AVAILABLE,1.0,39.4121,541266
8.885109360326481,-11.807273095439037,none,WD?,,PB 8382,7 .. 16,284.0145 .. 432.53094,0.0005666494 .. 1.844926e-12,0.028993921 .. 0.03350836,1129.3054,-1.0,-1.0,NOT_AVAILABLE,0.987006,3.2726,581995
357.1212025536532,-9.416754993209635,WD*,WD*,DA,GALEX J234829.1-092500,14 .. -1,16.348425 .. -1.0,1.6703574e-17 .. -1.0,0.13284327 .. --,403.82828,-1.0,-1.0,NOT_AVAILABLE,0.339561,4.7312,625693
318.34081907295393,-6.383251181607393,none,WD?,,Gaia DR3 6910932370192461312,29 .. 17,78.23779 .. 126.54944,0.012277861 .. 1.1401694e-07,0.010227647 .. 0.050328996,1083.541,0.0039228913,0.7665074,NOT_AVAILABLE,0.978881,1.1039,187856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210.0014165294432,3.2144664581271694,none,WD*,DA,GALEX J140000.3+031251,17 .. -1,147.22968 .. -1.0,2.3525176e-13 .. -1.0,0.052642535 .. --,-1.0,-1.0,-1.0,none,-1.0,-1.0,1521738
